In [1]:
#!/usr/bin/env python3.4

import os
import sys
import re
import uuid
import random
import imageio
import numpy as np
import torch
import torch_scatter

from scipy import misc
from PIL import Image
from tqdm import tqdm
from IPython import display

import matplotlib.pyplot as plt

def as_gif(images, path="temp.gif"):
  # Render the images as the gif (15Hz control frequency):
  images[0].save(path, save_all=True, append_images=images[1:], duration=int(1000/15), loop=0)
  gif_bytes = open(path,"rb").read()
  return gif_bytes

def read(file):
    if file.endswith('.float3'): return readFloat(file)
    elif file.endswith('.flo'): return readFlow(file)
    elif file.endswith('.ppm'): return readImage(file)
    elif file.endswith('.pgm'): return readImage(file)
    elif file.endswith('.png'): return readImage(file)
    elif file.endswith('.jpg'): return readImage(file)
    elif file.endswith('.pfm'): return readPFM(file)[0]
    else: raise Exception('don\'t know how to read %s' % file)

def write(file, data):
    if file.endswith('.float3'): return writeFloat(file, data)
    elif file.endswith('.flo'): return writeFlow(file, data)
    elif file.endswith('.ppm'): return writeImage(file, data)
    elif file.endswith('.pgm'): return writeImage(file, data)
    elif file.endswith('.png'): return writeImage(file, data)
    elif file.endswith('.jpg'): return writeImage(file, data)
    elif file.endswith('.pfm'): return writePFM(file, data)
    else: raise Exception('don\'t know how to write %s' % file)

def readPFM(file):
    file = open(file, 'rb')

    color = None
    width = None
    height = None
    scale = None
    endian = None

    header = file.readline().rstrip()
    if header.decode("ascii") == 'PF':
        color = True
    elif header.decode("ascii") == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode("ascii"))
    if dim_match:
        width, height = list(map(int, dim_match.groups()))
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().decode("ascii").rstrip())
    if scale < 0: # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>' # big-endian

    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)

    data = np.reshape(data, shape)
    data = np.flipud(data)
    return data, scale

def writePFM(file, image, scale=1):
    file = open(file, 'wb')

    color = None

    if image.dtype.name != 'float32':
        raise Exception('Image dtype must be float32.')

    image = np.flipud(image)

    if len(image.shape) == 3 and image.shape[2] == 3: # color image
        color = True
    elif len(image.shape) == 2 or len(image.shape) == 3 and image.shape[2] == 1: # greyscale
        color = False
    else:
        raise Exception('Image must have H x W x 3, H x W x 1 or H x W dimensions.')

    file.write('PF\n' if color else 'Pf\n'.encode())
    file.write('%d %d\n'.encode() % (image.shape[1], image.shape[0]))

    endian = image.dtype.byteorder

    if endian == '<' or endian == '=' and sys.byteorder == 'little':
        scale = -scale

    file.write('%f\n'.encode() % scale)

    image.tofile(file)

def readFlow(name):
    if name.endswith('.pfm') or name.endswith('.PFM'):
        return readPFM(name)[0][:,:,0:2]

    f = open(name, 'rb')

    header = f.read(4)
    if header.decode("utf-8") != 'PIEH':
        raise Exception('Flow file header does not contain PIEH')

    width = np.fromfile(f, np.int32, 1).squeeze()
    height = np.fromfile(f, np.int32, 1).squeeze()

    flow = np.fromfile(f, np.float32, width * height * 2).reshape((height, width, 2))

    return flow.astype(np.float32)

def readImage(name):
    if name.endswith('.pfm') or name.endswith('.PFM'):
        data = readPFM(name)[0]
        if len(data.shape)==3:
            return data[:,:,0:3]
        else:
            return data

    return imageio.imread(name)

def writeImage(name, data):
    if name.endswith('.pfm') or name.endswith('.PFM'):
        return writePFM(name, data, 1)

    return misc.imsave(name, data)

def writeFlow(name, flow):
    f = open(name, 'wb')
    f.write('PIEH'.encode('utf-8'))
    np.array([flow.shape[1], flow.shape[0]], dtype=np.int32).tofile(f)
    flow = flow.astype(np.float32)
    flow.tofile(f)

def readFloat(name):
    f = open(name, 'rb')

    if(f.readline().decode("utf-8"))  != 'float\n':
        raise Exception('float file %s did not contain <float> keyword' % name)

    dim = int(f.readline())

    dims = []
    count = 1
    for i in range(0, dim):
        d = int(f.readline())
        dims.append(d)
        count *= d

    dims = list(reversed(dims))

    data = np.fromfile(f, np.float32, count).reshape(dims)
    if dim > 2:
        data = np.transpose(data, (2, 1, 0))
        data = np.transpose(data, (1, 0, 2))

    return data

def writeFloat(name, data):
    f = open(name, 'wb')

    dim=len(data.shape)
    if dim>3:
        raise Exception('bad float file dimension: %d' % dim)

    f.write(('float\n').encode('ascii'))
    f.write(('%d\n' % dim).encode('ascii'))

    if dim == 1:
        f.write(('%d\n' % data.shape[0]).encode('ascii'))
    else:
        f.write(('%d\n' % data.shape[1]).encode('ascii'))
        f.write(('%d\n' % data.shape[0]).encode('ascii'))
        for i in range(2, dim):
            f.write(('%d\n' % data.shape[i]).encode('ascii'))

    data = data.astype(np.float32)
    if dim==2:
        data.tofile(f)

    else:
        np.transpose(data, (2, 0, 1)).tofile(f)


def RGBD2PCD(rgbs, depths, intrinsics, c2ws):
    # Assuming rgbs is of shape (N, 3, H, W), depths is of shape (N, 1, H, W), and c2ws is of shape (N, 4, 4)
    N, _, H, W = rgbs.shape
    if len(intrinsics.shape) == 2:
        intrinsics = intrinsics[None]
    intrinsics = torch.tensor(intrinsics, dtype=torch.float32, device=rgbs.device)

    with torch.no_grad():
        # Create meshgrid for x and y coordinates
        pos_x, pos_y = torch.meshgrid(torch.arange(W, device=rgbs.device), torch.arange(H, device=rgbs.device), indexing='xy')
        pos_x = pos_x.unsqueeze(0).expand(N, -1, -1)  # Shape: (N, H, W)
        pos_y = pos_y.unsqueeze(0).expand(N, -1, -1)  # Shape: (N, H, W)

        # Stack x and y coordinates and reshape to (N, H*W, 2)
        p_img = torch.stack([pos_x, pos_y], dim=-1).reshape(N, -1, 2)  # Shape: (N, H*W, 2)

        # Compute x_cam and y_cam
        x_cam = (p_img[:, :, 0] - intrinsics[:, 0, 2].unsqueeze(1)) * depths.reshape(N, -1) / intrinsics[:, 0, 0].unsqueeze(1)
        y_cam = (p_img[:, :, 1] - intrinsics[:, 1, 2].unsqueeze(1)) * depths.reshape(N, -1) / intrinsics[:, 1, 1].unsqueeze(1)

        # Stack x_cam, y_cam, depth, and ones to form homogeneous coordinates
        p_cam_homo = torch.stack([x_cam, y_cam, depths.reshape(N, -1), torch.ones_like(x_cam, device=rgbs.device)], dim=-1)  # Shape: (N, H*W, 4)

        # Transform to blender coordinate system
        p_cam_homo[:, :, 1:3] *= -1

        # Transform to world coordinates
        p_world = torch.matmul(p_cam_homo, c2ws.transpose(-2, -1))[:, :, :3]  # Shape: (N, H*W, 3)
        # world_points_homo = torch.bmm(c2ws, p_cam_homo.transpose(1, 2))  # 形状为 (N, 4, H*W)
        # world_points_homo = world_points_homo.transpose(1, 2)  # 形状为 (N, H*W, 4)
        # p_world = world_points_homo[..., :3] / world_points_homo[..., 3:4]  # 形状为 (N, H*W, 3)

        # Reshape rgb to (N, H*W, 3)
        rgb_world = rgbs.permute(0, 2, 3, 1).reshape(N, -1, 3)  # Shape: (N, H*W, 3)
    
    return p_world, rgb_world

def voxelization(p_world, feats, voxel_size, xyz_min=None):
    with torch.no_grad():
        # automatically determine the voxel size
        N, P, C = p_world.shape
        p_world = p_world.reshape(N*P, C)
        feats = feats.reshape(N*P, -1)
        if xyz_min is None:
            xyz_min = torch.min(p_world.reshape(-1, 3), dim=0).values
        voxel_index = torch.div(p_world - xyz_min[None, :], voxel_size[None, :], rounding_mode='floor')
        voxel_coords = voxel_index * voxel_size[None, :] + xyz_min[None, :] + voxel_size[None, :] / 2

        new_coors, unq_inv, unq_cnt = torch.unique(voxel_coords, return_inverse=True, return_counts=True, dim=0)
        feat_mean = torch_scatter.scatter(feats, unq_inv, dim=0, reduce='mean')

        new_feats = feat_mean[unq_inv]

        return new_coors

/tmp/ipykernel_8771/454091335.py:13: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  from scipy import misc


In [2]:
def readCamIntrinsic(file):
    with open(file, 'r') as file:
        lines = file.readlines()
    
    resolution = list(map(int, lines[1].strip().split()))  # 图像分辨率 (w, h)
    fx, fy = map(float, lines[3].strip().split())         # 焦距 (fx, fy)
    cx, cy = map(float, lines[5].strip().split())         # 主点坐标 (cx, cy)
    
    # 构造内参矩阵
    K = np.array([
        [fx,  0, cx],
        [ 0, fy, cy],
        [ 0,  0,  1]
    ])
    
    return K

def quaternion_to_rotation_matrix(q):
    w, x, y, z = q
    return np.array([
        [1 - 2 * (y**2 + z**2), 2 * (x * y - z * w), 2 * (x * z + y * w)],
        [2 * (x * y + z * w), 1 - 2 * (x**2 + z**2), 2 * (y * z - x * w)],
        [2 * (x * z - y * w), 2 * (y * z + x * w), 1 - 2 * (x**2 + y**2)]
    ])

def parse_visim_file(file_path):
    extrinsics_dict = {}

    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        # 跳过注释行
        if line.startswith("#") or line.strip() == "":
            continue
        
        # 解析每一行数据
        data = line.strip().split(",")
        timestamp = int(data[0])  # 时间戳
        position = np.array([float(data[1]), float(data[2]), float(data[3])])  # 平移向量
        quaternion = np.array([float(data[4]), float(data[5]), float(data[6]), float(data[7])])  # 四元数
        
        # 将四元数转换为旋转矩阵
        rotation_matrix = quaternion_to_rotation_matrix(quaternion)
        
        # 构造外参矩阵 [R | t]
        extrinsic_matrix = np.eye(4)
        extrinsic_matrix[:3, :3] = rotation_matrix
        extrinsic_matrix[:3, 3] = position
        #extrinsic_matrix = np.linalg.inv(extrinsic_matrix)
        
        # 将时间戳和外参矩阵存入字典
        extrinsics_dict[timestamp] = extrinsic_matrix

    return extrinsics_dict 

def normalize(v):
    return v / np.linalg.norm(v)

def parse_render_file(file_path):
    extrinsics_dict = {}

    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        # 跳过注释行
        if line.startswith("#") or line.strip() == "":
            continue
        
        # 解析每一行数据
        data = line.strip().split(" ")
        timestamp = int(data[0])  # 时间戳
        camera_pose = np.array([float(data[1]), float(data[2]), float(data[3])])  # 平移向量
        lookat_pose = np.array([float(data[4]), float(data[5]), float(data[6])])  # 四元数
        up = np.array([float(data[7]), float(data[8]), float(data[9])])  # 四元数
        R = np.diag(np.ones(4))
        R[2, :3] = normalize(lookat_pose - camera_pose)
        R[0, :3] = normalize(np.cross(R[2, :3], (up - camera_pose)))
        R[1, :3] = -normalize(np.cross(R[0, :3], R[2, :3]))
        T = np.diag(np.ones(4))
        T[:3, 3] = -camera_pose
        extrinsics_dict[timestamp] = np.linalg.inv(R.dot(T))
    return extrinsics_dict

In [4]:
data_folder = "../data/interiornet"
folder_seq_mapping = {}
with open(os.path.join(data_folder, "InteriorNetFiles.txt"), 'r') as f:
    lines = f.readlines()
# each line is like HD1/3FO4JXIK2PXE.zip
for line in lines:
    folder, seq = line.strip().split("/")
    if folder not in folder_seq_mapping:
        folder_seq_mapping[folder] = []
    folder_seq_mapping[folder].append(seq.split('.')[0])

# iterate over different folders and find sequence with maximum area
for folder, seqs in folder_seq_mapping.items():
    if folder == "HD7":
        continue
    max_area = 0
    max_seq = None
    for seq in tqdm(seqs, desc=f"Processing {folder}"):
        extrinsics_dict = parse_visim_file(os.path.join(data_folder, "GroundTruth_HD1-HD6", seq, f"velocity_angular_7_7", "cam0_gt.visim"))
        timestamps = list(extrinsics_dict.keys())
        cam_pos = []
        for i, timestamp in enumerate(timestamps):
            extrinsic_matrix = extrinsics_dict[timestamp]
            camera_position = extrinsic_matrix[:3, 3]
            cam_pos.append(camera_position)

        cam_pos = np.stack(cam_pos)
        area = (np.max(cam_pos[:, 0]) - np.min(cam_pos[:, 0])) * (np.max(cam_pos[:, 1]) - np.min(cam_pos[:, 1]))
        if area > max_area:
            max_area = area
            max_seq = seq
    print(f"Folder: {folder}, Sequence: {max_seq}, Area: {max_area}")

Processing HD1:   0%|          | 0/117 [00:00<?, ?it/s]

Processing HD1: 100%|██████████| 117/117 [00:01<00:00, 76.29it/s]


Folder: HD1, Sequence: 3FO4JXN0A5L3, Area: 72.64607122674983


Processing HD2: 100%|██████████| 119/119 [00:01<00:00, 78.67it/s]


Folder: HD2, Sequence: 3FO4KFWYFQW8, Area: 66.78944776799045


...

Folder: HD4, Sequence: 3FO4KDJXMRCO, Area: 92.95624621358384


Processing HD5: 100%|██████████| 116/116 [00:01<00:00, 77.96it/s]


Folder: HD5, Sequence: 3FO4KA19FAJI, Area: 54.85997065112974


Processing HD6: 100%|██████████| 108/108 [00:01<00:00, 78.81it/s]

Folder: HD6, Sequence: 3FO4JVTCFV05, Area: 108.02709203460294
